# Deutsch-Jozsa with 2 qubits
$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

In [1]:
from pyquil.quil import Program
from pyquil.gates import *
from pyquil import get_qc
import numpy as np
import pandas as pd

In [2]:
def create_dj_circuit(oracle, qubit_list):
    n_qubit = len(qubit_list)
    circuit = Program()
    circuit += X(qubit_list[-1])
    for i in range(n_qubit):
        circuit += H(qubit_list[i])
    for gate in oracle:
        circuit += gate
    for i in range(n_qubit):
        circuit += H(qubit_list[i])    
    ro = circuit.declare('ro', 'BIT', n_qubit)
    for i in range(n_qubit - 1):
        circuit += MEASURE(qubit_list[i], ro[i])
    return circuit

In [3]:
def run_circuit(circuit, qubit_list, qvm):    
    n = len(qubit_list) - 1
    exp_array = np.array([])
    for i in range(n):
        exp_array = np.append(exp_array,2**i)
    shots = 10**4
    circuit.wrap_in_numshots_loop(shots)
    #qc = get_qc('3q-qvm')  # You can make any 'nq-qvm' this way for any reasonable 'n'
    qc = get_qc("Aspen-M-1", as_qvm=qvm, execution_timeout=60)
    executable = qc.compile(circuit)
    result = qc.run(executable)
    bitstrings = result.readout_data.get('ro')[:,:n]
    results = bitstrings.dot(exp_array)
    unique_elements, counts_elements = np.unique(results, return_counts=True)
    print(pd.DataFrame(counts_elements, index = unique_elements,columns=["N"]))

10,000 runs. The states are:

|Index|Quantum State|
|:---------:|:---------:|
|0|\|00>|
|1|\|01>|
|2|\|10>|
|3|\|11>|

Best qubit in term of fidelity on the Aspen-M-1 Rigetti Processor:

In [4]:
qubit_list = [35,21,36] #Aspen-M-1

## Constant functions

\begin{align}
f(x_0,x_1) = 0\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{0}\ket{x} = \ket{x}
\end{align}


In [5]:
oracle = []
circuit = create_dj_circuit(oracle, qubit_list)

In [6]:
run_circuit(circuit, qubit_list, qvm = True)

         N
0.0  10000


In [7]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0  9839
1.0    51
2.0   110


\begin{align}
f(x_0,x_1) = 1\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{1}\ket{x} = -\ket{x}
\end{align}

In [8]:
oracle = [X(qubit_list[0])]
circuit = create_dj_circuit(oracle, qubit_list)

In [9]:
run_circuit(circuit, qubit_list, qvm = True)

         N
0.0  10000


In [10]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0  9823
1.0    53
2.0   123
3.0     1


# Balanced Functions

\begin{align}
f(x_0,x_1) = x_0\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{x_0}\ket{x}
\end{align}

In [11]:
oracle = [CNOT(qubit_list[0],qubit_list[2])]
circuit = create_dj_circuit(oracle, qubit_list)

In [12]:
run_circuit(circuit, qubit_list, qvm = True)

         N
1.0  10000


In [13]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0   683
1.0  9123
2.0    28
3.0   166


\begin{align}
f(x_0,x_1) = x_1\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{x_1}\ket{x}
\end{align}

In [14]:
oracle = [CNOT(qubit_list[1],qubit_list[2])]
circuit = create_dj_circuit(oracle, qubit_list)

In [15]:
run_circuit(circuit, qubit_list, qvm = True)

         N
2.0  10000


In [16]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0  1893
1.0    21
2.0  8038
3.0    48


\begin{align}
f(x_0,x_1) = x_0 + x_1\;\;\; \Rightarrow \;\;\; U_f\ket{x}=(-1)^{(x_0+x_1)}\ket{x}
\end{align}

In [17]:
oracle = [CNOT(qubit_list[0],qubit_list[2]),CNOT(qubit_list[1],qubit_list[2])]
circuit = create_dj_circuit(oracle, qubit_list)

In [18]:
run_circuit(circuit, qubit_list, qvm = True)

         N
3.0  10000


In [19]:
run_circuit(circuit, qubit_list, qvm = False)

        N
0.0   127
1.0  1768
2.0   242
3.0  7863
